# Requirements and Basic Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#! pip install openai
!pip install -q -U agno
!pip install -q chromadb
!pip install -q qdrant-client
!pip install -q pypdf
!pip install -q -U ddgs

In [ ]:
from pprint import pprint
import shutil
import random, os
os.environ['OPENAI_API_KEY'] = ''

from agno.agent import Agent
from agno.team import Team
from agno.models.openai import OpenAIChat

from agno.db.sqlite import SqliteDb

from tqdm import tqdm

from agno.knowledge.knowledge import Knowledge
from agno.knowledge.reader.pdf_reader import PDFReader # chunking
from agno.knowledge.embedder.openai import OpenAIEmbedder
from agno.vectordb.qdrant import Qdrant
from agno.tools.duckduckgo import DuckDuckGoTools

import gradio as gr
import google.colab.output

# Paths and Databases

In [ ]:
# DRIVE files
DRIVE_PATH = "/content/drive/Othercomputers/Il mio MacBook Pro/My notebooks/Rizzo AI Academy/Agenti in Python/Gestione FInanaziaria Aziendale"
FILE_NAMES = ["appunti_corso.pdf"]
DRIVE_FILES = [os.path.join(DRIVE_PATH, FILE_NAMES[i]) for i in range(len(FILE_NAMES))]
DRIVE_QDRANT = os.path.join(DRIVE_PATH, "qdrant_db")
DRIVE_AGNO = os.path.join(DRIVE_PATH, "agno.db")

# LOCAL files
LOCAL_PATH = "/content/database_locali"
os.makedirs(LOCAL_PATH, exist_ok = True)
LOCAL_FILES = [os.path.join(LOCAL_PATH, FILE_NAMES[i]) for i in range(len(FILE_NAMES))]
LOCAL_QDRANT = os.path.join(LOCAL_PATH, "qdrant_db")
LOCAL_AGNO = os.path.join(LOCAL_PATH, "agno.db")

# copy DRIVE -> LOCAL
if os.path.exists(DRIVE_QDRANT):
  if os.path.exists(LOCAL_QDRANT):
    shutil.rmtree(LOCAL_QDRANT)
  shutil.copytree(DRIVE_QDRANT, LOCAL_QDRANT)

if os.path.exists(DRIVE_AGNO):
  shutil.copy(DRIVE_AGNO, LOCAL_AGNO)

for drive_file, local_file in zip(DRIVE_FILES, LOCAL_FILES):
  if os.path.exists(drive_file):
    shutil.copy(drive_file, local_file)

def backup_to_drive():
  print("Salvataggio su Drive in corso, non spegnere...")
  try:
    if os.path.exists(DRIVE_QDRANT):
      shutil.rmtree(DRIVE_QDRANT)
    shutil.copytree(LOCAL_QDRANT, DRIVE_QDRANT)
    shutil.copy(LOCAL_AGNO, DRIVE_AGNO)
    print("Backup completato con successo!")
  except Exception as e:
    print(f"Errore durante il backup: {e}")

# Embedder

In [ ]:
embedder = OpenAIEmbedder(id = "text-embedding-3-small")

import gc
if 'vector_db' in locals():
  try:
    vector_db.client.close()
    print("🔒 Client Qdrant chiuso correttamente.")
  except:
    pass
  del vector_db

# 2. Forza la pulizia della memoria
gc.collect()

vector_db = Qdrant(collection = "prova",
                   path = LOCAL_QDRANT,
                   embedder = embedder)

num_vectors = 0
try:
  info = vector_db.client.get_collection("prova")
  num_vectors = info.points_count
except Exception:
  num_vectors = 0

agno_db = SqliteDb(db_file = LOCAL_AGNO)

knowledge = Knowledge(vector_db = vector_db)

reader = PDFReader(chunk_size = 1000,
                   split_on_pages = True)

if not os.path.exists(LOCAL_QDRANT) or not os.listdir(LOCAL_QDRANT) or num_vectors == 0:
  print("📥 Knowledge base vuota. Inserimento documenti in corso...")
  for path in tqdm(LOCAL_FILES, desc = "Inserimento documenti"):
    if os.path.exists(path):
      knowledge.insert(path=path, reader = reader)

  backup_to_drive()
else:
  print(f"🚀 KB caricata. Vettori trovati: {num_vectors}")

🔒 Client Qdrant chiuso correttamente.
🚀 KB caricata. Vettori trovati: 188


# Agents and Team definition

In [ ]:
id = "gpt-4o-mini"

google_agent = Agent(
    name = "google_agent",
    model = OpenAIChat(id = id),
    role = "Esegue ricerche online e trova informazioni aggiornate",
    description = "Sei un esperto di ricerche online. Il tuo compito è trovare informazioni aggiornate su internet.",
    instructions = ["Fornisci solo informazioni verificate", "Cita le fonti e metti il link se possibile."],
    expected_output = "Una risposta con al massimo 5 frasi, usando un gergo tecnico.",
    tools = [DuckDuckGoTools()])

rag_agent = Agent(
    name = "rag_agent",
    model = OpenAIChat(id = id),
    role = "Assistente finanziario che risponde usando la knowledge base",
    knowledge = knowledge,
    search_knowledge = True,
    description = "Sei un esperto di Economia e Gestione Finanaziaria Aziendale.",
    instructions = ["Rispondi in modo chiaro ma conciso.", "Riporta le pagine del PDF da cui hai preso le informazioni."],
    expected_output = "Una risposta con al massimo 5 frasi, usando lo stesso gergo tecnico del pdf.",
    )

team = Team(
    name = "financial_assistant",
    model = OpenAIChat(id = id),
    members = [google_agent, rag_agent],
    respond_directly = False, # il team leader elabora le risposte dei membri
    db = agno_db,
    description = """Sei Supernotes, l'assistente finanziario personale di Elena. La aiuterai a risolvere
                     dubbi sul materiale di esame di 'Gestione Finanaziaria Aziendale'.""",
    instructions = ["Alla prima interazione, presentati e saluta cordialmente Elena.",
                    "FASE 1: Cerca nella knowledge base usando il 'rag_agent'. Se hai trovato la risposta, rispondi secondo le indicazioni.",
                    "FASE 2: Se la domanda non è di tipo economico, scusati cordialmente dicendo che non sei esperto in materia.",
                    "FASE 3: Se la domanda è di tipo economico e non presente nella knowledge base, delega ricerche web al 'google_agent'.",
                    "Se Elena saluta, augurale buona giornata e dille che è la migliore."],
    expected_output = "Una risposta con al massimo 10 frasi, usando lo stesso gergo tecnico del pdf.",
    enable_user_memories = True,
    add_history_to_context = True,
    num_history_runs = 30,
    stream = True,
    markdown = True
)

In [ ]:
def chat_assistant(messaggio, storia):
  response = team.run(messaggio, stream = False)
  backup_to_drive()
  return response.content

demo = gr.ChatInterface(
  fn = chat_assistant,
  type = "messages",
  title = "Supernotes 🥰 - il tuo assistente di Gestione Finanziaria Aziendale 🤓",
  theme = "soft"
)

# App/Gradio Interface

In [ ]:
demo.launch(share = True, debug = False, inbrowser = True, inline=False)

if demo.share_url:
    # ignore_result=True evita che Colab aspetti un feedback dal browser
    google.colab.output.eval_js(f'window.open("{demo.share_url}", "_blank")', ignore_result=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://386ea66008d3b72f1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
